### Imports

In [19]:
#imports
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import sys
import os
import pandas as pd
import time
import math
import random

### Initialisation Functions

In [20]:
def calculate_minimum_trucks_from_demand(demand_from_nodes, avg_vehicle_capacity):
    total = 0
    with open(demand_from_nodes, 'r') as f:
        for line in f:
            total = total + int(line)
        f.close()
    
    minimum_num = math.floor(total / avg_vehicle_capacity)
    return minimum_num

def vehicle_demand_initialiser(total_number_of_nodes, demand_from_nodes):
    with open(demand_from_nodes, 'w') as f:
        f.write("0" + '\n')
        for i in range(total_number_of_nodes - 1):
            demand = random.randint(1, 10)
            f.write(str(demand) + '\n')
        f.close()
        
def vehicle_time_window_initialiser(total_number_of_nodes, nodes_time_window):
    
    with open(nodes_time_window, 'w') as f:
        f.write("1"+ ' ' + "300" + '\n')
        for i in range(total_number_of_nodes - 1):
            begin = random.randint(1,10)
            end   = random.randint(begin+1, 100)
            f.write(str(begin) + ' ' + str(end) + '\n')
        f.close()

In [21]:
def format_demand_data():
    demands = []
    with open('data/Node_Demands.txt', 'r') as f:
        for line in f:
            demands.append(int(line))
        f.close()
    demands = demands[1:] #removing depot node having zero demand
    
    with open('data/demand_delivery_contraint.csv', 'w') as f:
        f.write("Store" + "," + "Demand" + "," + "Acceptance constraint" + "\n")
        for i in range(len(demands)):
            f.write("S" + str(i) + "," + str(demands[i]) + "," + str(float(demands[i]/2)) + "\n")

In [22]:
def format_obtained_routes(number_of_stations, obtained_routes = 'output/route.txt'):
    
    #Maximum number of stops in header of table
    max_num_of_stops = 10
    
    with open('data/final_routing_table.csv', 'w') as f:
        #creating the header of table
        f.write("Routes" + "," + "Stop_1" + ",")
        for i in range(2, max_num_of_stops):
            f.write("Stop_" + str(i) + ",")
        f.write("\n")
        
        for i in range(number_of_stations):
            f.write(str(i) + "," + "S" + str(i) + "\n")
        
        f.close()
    
    with open(obtained_routes, 'r') as f:
        current_station = number_of_stations #just a counter
        
        for line in f:
            # Ignore rest, select only the route from the output
            if(line.split()[0][0] != '0'):
                continue
            
            split = line.split('->')
            route = [int(i) for i in split[1:-1]]
            
            #creating a string to append to the final routing csv file 
            route_string = ''
            for stop in route:
                route_string += 'S' + str(stop) + ','
            route_string += '\n'
            
            with open('data/final_routing_table.csv', 'a') as f2:
                f2.write(str(current_station) + "," + route_string)
                current_station += 1
                f2.close()
        f.close()

### Data Model

In [23]:
def create_data_model(demand_from_nodes, nodes_time_window):
    """Stores the data for the problem."""

    ### INITIALISE DATA STRUCTURE
    data = {}
    data['demands'] = []
    data['time_matrix']  = []
    data['time_windows'] = []
    data['num_vehicles'] = 2
    data['depot'] = 0
    data['vehicle_capacities'] = [50] * data['num_vehicles'] #Modified later on, shouldn't be a concern here.


    ### READ DEMAND FOR EACH NODE FROM FILE
    with open(demand_from_nodes, 'r') as f:
        for line in f:
            data['demands'].append(int(line.rstrip()))
        f.close()
    
    number_of_nodes = len(data['demands'])
    ### READ TRAVEL TIME DATA FROM FILE
    time_matrix_data = pd.read_csv('data/site_time.csv')
    time_matrix_data = time_matrix_data.iloc[:number_of_nodes, 1:number_of_nodes+1]
    time_matrix_data[time_matrix_data.columns[0]] = 0
    time_matrix      = time_matrix_data.values.tolist()
    for site_time_data_row in time_matrix:
        data['time_matrix'].append(site_time_data_row)
    
    ### READ TIME WINDOW DATA FROM FILE
    with open(nodes_time_window, 'r') as f:
        for line in f:
            window_start, window_end = map(int,line.rstrip().split())
            data['time_windows'].append((window_start, window_end))
        f.close()
        
    return data


### Solution Printing Fn

In [24]:
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += '{}->'.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Time of the route: {}m\n'.format(route_distance)
        print(plan_output)
        with open('output/route.txt', 'a') as f:
            f.write(plan_output)
        
        total_distance += route_distance
    print('Total Time of all routes: {}m'.format(total_distance))


### Methodolgy Wrapper Function

In [34]:
def wrapper(number_of_vehicles, vehicle_capacity, demand_from_nodes, nodes_time_window):
    
    # Instantiate the data problem.
    data = create_data_model(demand_from_nodes, nodes_time_window)
    data['num_vehicles'] = number_of_vehicles
    data['vehicle_capacities'] = [vehicle_capacity] * number_of_vehicles


    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

        
    # Time Callback and constraints
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]
    
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    
    # Using time transit callback as optimisation parameter
    routing.SetArcCostEvaluatorOfAllVehicles(time_callback_index)

    
    ### Smaller values limit the travel of vehicles, the following values have no affect on time windows
    ### Large values makes sure that we get the best solution for now
    '''Time Window Constraint'''
    time = 'Time'
    routing.AddDimension(
        time_callback_index,
        10000,  # allow waiting time
        10000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    
    ''' 
    Limiting the number of stops travelled by each vehicle
    We set an upper bound of one of the routing dimensions (here, time dimension)
    No vehicle is allowed to travel more than the specified units for that dimension
    
    We can use distance, time, or any other dimension for setting the upper bound.
    
    Please note: minimum value of upper bound = max(distance(depot, node))
    Otherwise solution will not exist, as at least one node would become unreachable
    '''
    for vehicle_id in range(data['num_vehicles']):
        time_dimension.SetSpanUpperBoundForVehicle(10, vehicle_id) 


    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        # Require that a vehicle must visit a location during the location's time window.
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
    
    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))
    
    
    
    # Demand callback and constaints
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')


    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        if(os.path.exists('output/route.txt')):
            os.remove('output/route.txt')

        print_solution(data, manager, routing, solution)
        print("Vehicles Required: {}".format(number_of_vehicles))
        print("-"*40)
        return 1
    else:
        print("Current Number of Vehicles = {}".format(number_of_vehicles))
        print("No Solution Yet")
        print("-"*40)
        print("Incrementing number of vehicles")
        return 0

### Main Function

In [35]:
def main():
    
    """ Files Required
            - site_time.csv (time to travel between any two nodes)
            - Node_Demands.txt (can be generated)
            - nodes_time_window.txt (can be generated)
            
            To change number of nodes, just edit the variable below and regenerate demand and time windows.
            Solution isn't guaranteed when new time windows are created
            
            Preferably change the name of text file so that previous data isn't lost.
    """
    
    
    demand_from_nodes  = "data/Node_Demands.txt"
    nodes_time_window  = "data/nodes_time_window.txt" # Filename containing time window for each node
    number_of_nodes    = 20
    
#     vehicle_demand_initialiser(number_of_nodes, demand_from_nodes)
#     vehicle_time_window_initialiser(number_of_nodes, nodes_time_window)
    
    vehicle_capacity   = 30
    current_num_of_trucks = calculate_minimum_trucks_from_demand(demand_from_nodes, vehicle_capacity)
    print("minimum_num_of_vehicles = {}".format(current_num_of_trucks) + '\n' + '-'*40)
    solution = wrapper(current_num_of_trucks, vehicle_capacity, demand_from_nodes, nodes_time_window)
    
    cntr = 0
    while True:
        current_num_of_trucks += 1
        cntr += 1
        solution = wrapper(current_num_of_trucks, vehicle_capacity, demand_from_nodes, nodes_time_window)
        if(current_num_of_trucks > 100):
            break
        if(cntr == 10):
            break
    
    #TODO: Add binary search instead of linear search.
    
    
    '''
    Optimisation Inventory Routing Problem Part:
    
     Stations = Total nodes - depot
    
    '''
    format_demand_data()
    format_obtained_routes(number_of_nodes - 1, obtained_routes = 'output/route.txt')

In [36]:
if __name__ == '__main__':
    main()

minimum_num_of_vehicles = 4
----------------------------------------
Current Number of Vehicles = 4
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 5
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 6
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 7
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 8
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 9
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 10
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Current Number of Vehicles = 11
No Solution Yet
----------------------------------------
I

#### Some Results
Upper Bound = 12m | Min Vehicles 5 | Min time = 46m

Upper Bound = 11m | Min Vehicles 6 | Min time = 49m

# Inventory Routing Formulation 


<b>Decision Variable:</b>

\begin{equation}
\begin{array}{11}
\ Y_{i,k} & \forall i \in R, k \in WD
\end{array}
\end{equation}
\begin{equation}
\begin{array}{11}
\ X_{i,j,k} & \forall i \in R, j \in S, k \in WD
\end{array}
\end{equation}

<b>Objective Function:</b>

\begin{equation}
\begin{array}{ll}
\text{minimise} &  \sum_{i \in R}  \sum_{k \in W} Y_{i,k}  c_{i,k}\\
\end{array}
\end{equation}


<b>Subject to:</b>
\
<i>Respecting Demand Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{i \in R} \sum_{k \in W}  X_{i,j,k} m_{i,j} = d_j & \forall j \in S
\end{array}
\end{equation}

\
<i>Respecting Delivery Acceptance Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{i \in R} X_{i,j,k} m_{i,j} \le a_j & \forall j \in S, k \in W
\end{array}
\end{equation}

\
<i>Respecting Truck Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{j \in S} X_{i,j,k}m_{i,j} \le C_iY_{i,k} & \forall i \in R, k \in W
\end{array}
\end{equation}



\
<b>General Representations</b>

\
<i>Sets</i>
* <i>R</i>: Set of routes
* <i>S</i>: Set of stores
* <i>W</i>: Set of days in a week

\
<i>Parameters</i>
\
$
\begin{equation} 
\begin{array}{11}
d_j & \text{: Weekly demand of store j,} & j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
a_j & \text{: Delivery acceptance limits for store j,} & j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
m_{i,j} & \text{: Dummy variable to indicate 1 if store j is along route i and 0 otherwise,} & i \in R, j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
c_{i,k} & \text{: Cost of  truck on each route i on day k } 
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
C_{i} & \text{: Full capacity of truck on route i on each day  } 
\end{array}
\end{equation}
$

\
<i>Variables</i>
$
\begin{equation} 
\begin{array}{11}
X_{i,j,k} & \text{: Units on each route i to be delivered to a store j on day k} \\
Y_{i,k} & \text{: If route i is used on day k}
\end{array}
\end{equation}
$

\
<i>Entities</i>
* <i>i</i>: Route in R
* <i>j</i>: Store in S
* <i>k</i>: Day in W


In [45]:
#Importing Packages

from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

In [46]:
#Importing Data

routes = pd.read_csv("data/final_routing_table.csv").to_numpy()
print(routes)

demand = pd.read_csv("data/demand_delivery_contraint.csv").to_numpy()
print(demand)


[[0 'S0' nan nan nan nan nan nan nan nan nan]
 [1 'S1' nan nan nan nan nan nan nan nan nan]
 [2 'S2' nan nan nan nan nan nan nan nan nan]
 [3 'S3' nan nan nan nan nan nan nan nan nan]
 [4 'S4' nan nan nan nan nan nan nan nan nan]
 [5 'S5' nan nan nan nan nan nan nan nan nan]
 [6 'S6' nan nan nan nan nan nan nan nan nan]
 [7 'S7' nan nan nan nan nan nan nan nan nan]
 [8 'S8' nan nan nan nan nan nan nan nan nan]
 [9 'S9' nan nan nan nan nan nan nan nan nan]
 [10 'S10' nan nan nan nan nan nan nan nan nan]
 [11 'S11' nan nan nan nan nan nan nan nan nan]
 [12 'S12' nan nan nan nan nan nan nan nan nan]
 [13 'S13' nan nan nan nan nan nan nan nan nan]
 [14 'S14' nan nan nan nan nan nan nan nan nan]
 [15 'S15' nan nan nan nan nan nan nan nan nan]
 [16 'S16' nan nan nan nan nan nan nan nan nan]
 [17 'S17' nan nan nan nan nan nan nan nan nan]
 [18 'S18' nan nan nan nan nan nan nan nan nan]
 [19 'S5' 'S3' 'S7' 'S9' 'S16' nan nan nan nan nan]
 [20 'S2' 'S4' 'S12' 'S14' nan nan nan nan nan nan]
 [21

In [47]:
#Demand
d = demand[:,1]

#Delivery constraints
dc = demand[:,2]

#Route matrix
#Store names 
snames = demand[:,0]

count = 1
num_routes = len(routes)
num_stores = len(demand)
mat = np.zeros(num_routes)
for i in snames:
    routestore1 = (routes[:,1] == str(i)) * 1
    routestore2 = (routes[:,2] == str(i)) * 1
    mainroutestore = routestore1 + routestore2
    mat = np.row_stack((mat, mainroutestore))
    #mat = np.concatenate((mat, mainroutestore),axis=1)
    
mat = np.array(mat, dtype='int16')[1:,:].transpose((1,0))

print(mat)


[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]]


In [48]:
#Shipping costs
c = np.full((num_routes, num_stores), 1000)
c

#Supply constraints (truck capacity)
t = np.full((num_routes,7), 30)
t

array([[30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30]])

In [49]:
#Transportation Problem
m4 = Model('transportation')

#Variables
#Variables are in proportion
var = m4.addVars(num_routes,num_stores,7)
yvar = m4.addVars(num_routes,7,vtype = GRB.BINARY)

#Objective
m4.setObjective(sum(yvar[i,k] for i in range(num_routes) for k in range(7)), GRB.MINIMIZE)

#Weekly Demand
for j in range(num_stores):
    m4.addConstr(sum(var[i,j,k]*mat[i,j] for i in range(num_routes) for k in range(7)) == d[j])
    
#Delivery Constraints
for j in range(num_stores):
    for k in range(7):
        m4.addConstr(sum(var[i,j,k]*mat[i,j] for i in range(num_routes)) <= 0.6*dc[j])

#Supply constraint
for i in range(num_routes):
    for k in range(7):
        m4.addConstr(sum(var[i,j,k]*mat[i,j] for j in range(num_stores)) <= t[i,k]*yvar[i,k])
        
#Solving the optimization problem
m4.optimize()

#Printing the optimal solutions obtained
print("Optimal Solutions:")
for i, val in var.items():
    if val.getAttr("x") != 0:
        print("Number of units from route %g to store %g on day %g:\t %g " %(i[0]+1, i[1]+1, i[2]+1, val.getAttr("x")))
        
        
#Printing y
for i, val in yvar.items():
    print("Run route %g on day %g:\t %g " %(i[0]+1, i[1]+1, val.getAttr("x")))
    
print(yvar)
 


Academic license - for non-commercial use only - expires 2021-05-10
Using license file /Users/adityagoel/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 320 rows, 3360 columns and 777 nonzeros
Model fingerprint: 0x9cfca9f0
Variable types: 3192 continuous, 168 integer (168 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+01]
Found heuristic solution: objective 75.0000000
Presolve removed 283 rows and 3311 columns
Presolve time: 0.04s
Presolved: 37 rows, 49 columns, 105 nonzeros
Found heuristic solution: objective 60.0000000
Variable types: 28 continuous, 21 integer (21 binary)

Root relaxation: objective 5.533333e+01, 35 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

In [50]:
for i in range(num_routes):
    for k in range(7):
        print(yvar[i,k].getAttr("x"))

-0.0
1.0
1.0
1.0
1.0
-0.0
-0.0
-0.0
1.0
-0.0
1.0
1.0
-0.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
-0.0
1.0
1.0
-0.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
1.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
-0.0
1.0
1.0
-0.0
1.0
-0.0
1.0
1.0
1.0
1.0
-0.0
-0.0
-0.0
1.0
-0.0
1.0
1.0
-0.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
-0.0
1.0
1.0
-0.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
-0.0
-0.0
1.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
-0.0
1.0
1.0
1.0
1.0
-0.0
-0.0
1.0
-0.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
1.0
1.0
-0.0
-0.0
1.0
-0.0
1.0
1.0


In [51]:
print("Finish")

Finish
